In [1]:
%run useful_functions.py

C:\Users\natha\Desktop\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\Work_Nathan\useful_functions.py:941: SyntaxWarning: invalid escape sequence '\s'
  item1A = re.sub('\s{2,}',' ', item1A)
C:\Users\natha\Desktop\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\Work_Nathan\useful_functions.py:952: SyntaxWarning: invalid escape sequence '\s'
  document = re.sub('\s{2,}',' ', document)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\natha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


update 16


In [2]:
directory = 'Financial_values/'

price_daily_dict={}
price_monthly_dict={}

for ticker in tqdm(os.listdir(directory)):
    if ticker=="TSLA" or True:
    
        file_path = 'Financial_values/'+ticker+'/daily_prices.csv'
    
        price_table = pd.read_csv(file_path)
    
        price_table['date'] = pd.to_datetime(price_table['date'])
        price_table.set_index('date', inplace=True)
        price_table.dropna(inplace=True)

        if price_table.shape[0]==0:
            print("EMPTY : "+ticker)
            

        else: #skip if the table is empty which happens sometimes
            
        
            #adjust by split factor
            price_table["prc"]/=price_table["cfacpr"]
            price_table["openprc"]/=price_table["cfacpr"]
            price_table["bidlo"]/=price_table["cfacpr"]
            price_table["askhi"]/=price_table["cfacpr"]
        
            price_table=price_table.drop('cfacpr', axis=1)
        
            #estimation of bid-ask
            price_table["log bid ask factor"]= price_table[['prc']].rolling(window='31D').apply(bid_ask_estimator, args=(price_table,)).values
        
        
            #complete the Nan or 0
            to_be_completed = price_table["log bid ask factor"].copy().values
            to_be_completed[to_be_completed < 0.000001] = np.nan
            first_float_value_is_behind = False
            first_float_value = np.nan
            
            for i, val in enumerate(to_be_completed.copy()):
                if first_float_value_is_behind:
                    if np.isnan(val):
                        to_be_completed[i] = to_be_completed[i - 1]
                else:
                    if not(np.isnan(val)):
                        first_float_value_is_behind = True
                        first_float_value = val
            
            to_be_completed[np.isnan(to_be_completed)] = first_float_value
        
            price_table["log bid ask factor"]=to_be_completed
        
            #remember that the estimator is based on logarithm of prices!
            price_table["plus_cost"]=price_table['prc']*np.exp(price_table["log bid ask factor"]*0.5)-price_table['prc']
            price_table["minus_cost"]=price_table['prc']*np.exp(-price_table["log bid ask factor"]*0.5)-price_table['prc']
        
            #price_table["buy price"]=price_table["plus_cost"]+price_table['prc']
            #price_table["sell price"]=price_table["minus_cost"]+price_table['prc']
        
            
    
            #price_daily_dict[ticker]=[np.mean(price_table["plus_cost"]/price_table["prc"]),-np.mean(price_table["minus_cost"]/price_table["prc"])]
    
            
            price_table_monthly=price_table[["plus_cost"]].resample('ME').mean()
            price_table_monthly["minus_cost"]=price_table[["minus_cost"]].resample('ME').mean().values
    
            
            
            price_table_monthly["plus_cost_last"]=price_table[["plus_cost"]].resample('ME').last().values
            price_table_monthly["minus_cost_last"]=price_table[["minus_cost"]].resample('ME').last().values
    
            price_table_monthly["prc"]=price_table[["prc"]].resample('ME').last().values
    
            price_monthly_dict[ticker]=[np.mean(price_table_monthly["plus_cost"]/price_table_monthly["prc"]),
                                      -np.mean(price_table_monthly["minus_cost"]/price_table_monthly["prc"]),
                                      np.mean(price_table_monthly["plus_cost_last"]/price_table_monthly["prc"]),
                                      -np.mean(price_table_monthly["minus_cost_last"]/price_table_monthly["prc"]),]


            price_table_monthly.to_csv('Financial_values/'+ticker+'/bid_ask_monthly.csv')
    

  0%|          | 0/7083 [00:00<?, ?it/s]

EMPTY : ADTI
EMPTY : AESC
EMPTY : AITX
EMPTY : APET
EMPTY : ARMC
EMPTY : ATDS
EMPTY : AXIM
EMPTY : BCRD
EMPTY : BENH
EMPTY : BLI
EMPTY : BRER
EMPTY : BRVO
EMPTY : BSEC
EMPTY : BSYN
EMPTY : BTN
EMPTY : CDEV
EMPTY : CGSI
EMPTY : CLCN
EMPTY : CLOW
EMPTY : CLTH
EMPTY : CNCL
EMPTY : COW
EMPTY : CSBB
EMPTY : DEWY
EMPTY : DION
EMPTY : DUCK
EMPTY : ERES
EMPTY : EXCL
EMPTY : FHLD
EMPTY : FICE
EMPTY : FLYD
EMPTY : GPCO
EMPTY : GRAY
EMPTY : GTCH
EMPTY : GWIN
EMPTY : HABE
EMPTY : HLCO
EMPTY : HRST
EMPTY : HYDI
EMPTY : IGNE
EMPTY : IMPE
EMPTY : INT
EMPTY : ISCO
EMPTY : KAYS
EMPTY : KENS
EMPTY : KITL
EMPTY : LCAR
EMPTY : LEXP
EMPTY : LVT
EMPTY : MAGE
EMPTY : MKRS
EMPTY : MLND
EMPTY : MMNT
EMPTY : MSOF
EMPTY : MSPC
EMPTY : MSTC
EMPTY : MTBC
EMPTY : MULT
EMPTY : NBCO
EMPTY : NLPI
EMPTY : NOVC
EMPTY : NRGD
EMPTY : NSTM
EMPTY : OAKR
EMPTY : OCEX
EMPTY : ODYY
EMPTY : OFC
EMPTY : PFDR
EMPTY : PGAI
EMPTY : PHOT
EMPTY : PPAC
EMPTY : PTCO
EMPTY : PULP
EMPTY : QBIO
EMPTY : RE
EMPTY : RMMI
EMPTY : SACC
EMPTY :